# Genetic Algorithm Assignment 1
- Student Name: **Vinicius Moura Barros**
- Student Number: **T00244396**

30% of the overall grade for this module

Marks indciated in sections below are based on percentage of marks allocated for this module

In this assignment you must choose a problem, and attempt to use the Genetic Alogrithm that we developed in class to solve this problem.





In [1]:
# Imports required for the assigment
import math
import random
from abc import abstractmethod
from copy import deepcopy
from dataclasses import dataclass
from typing import Any, Optional

## The Problem         **(~30%)**



---

The theme for this problem is a video game character building. The objective of the game, is to build a character (or just "char" or individual) that has the capabilities of defeating a given boss. The boss, is also a character, but with pre-defined attributes. We'll use Generic Algorithms to solve this selection problem.

### Character Attributes
Each character (including the boss) has 4 attributes:
- **Health Points (HP)**: Integer amount that represents life points of the character
- **Strenght (STR)**: The amount of damage the character deals when attacking
- **Defense (DEF)**: The amount of damage avoided, should the character gets attacked
- **Healing (HEAL)**: The amount of recovered HP after each turn

Each character starts with 20 points, which should be distributed between the 4 possible attributes. Exemple:
- Character 1: 5 HP, 5 STR, 5 DEF, 5 HEAL
- Character 2: 15 HP, 5 STR, 0 DEF, 0 HEAL
- Character 3: 1 HP, 19 STR, 0 DEF, 0 HEAL
- Character 4: 1 HP, 1 STR, 18 DEF, 0 HEAL

Constraints: 
- Each character should have at least 1 point in HP (or as defined in the problem)
- The sum of the stats of each character (except the boss) should be exactly 20 stats point (or as defined in the problem)

### Battle System
The battle system is based on turns, following the order:

- The character attacks the boss 
  - The boss will have its HP deducted
  - If the boss HP is gets equal or below 0, the character wins the game
- The boss attacks the character
  - The character will have its HP deducted
  - If the character HP is gets equal or below 0, the character loses the game
- The healing effect is triggered for the character and for the boss
- Repeat the process until character wins or loses game (or after 50 rounds) also considered a loss for the character.

---



##   Discussion of the suitablity of Genetic Algorithms


---

The genetic algorithm here is used to find what's the potential best combination of attributes for a character to have, so it can win a match against a boss. 
As there are multiple potential combinations of HP, STR, DEF and HEAL, the genetics algorithm seems appropriate to randomly find a combination that seems to fit and keep evolving it until a most appropriate combination is found. 

---



# The problem and the cost function   **(~20%)**

In [ ]:
class Problem:
  def __init__(self, boss):
    self.max_stats_points = 20
    self.max_fight_rounds = 50
    self.min_hp = 1
    self.boss = boss

  def generate_random_stats(self):
    """
    As the name suggests, this method generates random attributes
    that can be used to create a new invididual
    """
    # we need to make sure elements have at least 1 HP
    stats = [1, 0, 0, 0]  # HP, STR, DEF, HEAL
    for _ in range(self.max_stats_points -1): # -1 because of rule of min 1 HP
      stats[random.choice([0,1,2,3])] += 1

    return {
        'hp': stats[0],
        'strength': stats[1],
        'defense': stats[2],
        'healing': stats[3],
    }

  def gen_2_random_unique(self, min, max):
    """
    This method generated 2 random unique numbers, used to randomly find 
    parents for a new individual
    """
    n1 = random.randint(min, max)
    n2 = n1
    while n2 == n1:
      n2 = random.randint(min, max)

    return n1, n2
    
  def cost_function(self, fight):
    """
    This is the function that will calculate how fit an individual is.
    Basically it gets the result of a fight as a parameter to calculate 
    the cost of this individual.

    Example of expected output of a fight:
    { 
      'is_victory': bool,
      'rounds': int,
      'reason': str,
      'char': {'hp': -1, 'max_hp': 5, 'strength': 6, 'defense': 3, 'healing': 6},
      'boss': {'hp': 17, 'max_hp': 20, 'strength': 9, 'defense': 3, 'healing': 2}
    }
    
    The Best individual is the one that:
    - wins the fight against the boss
    - takes less rounds to win
    - or if individual loses, the one that deals more damage to the boss is considered better

    Mathematically, the cost can be represented as:
    (0 points per win) + (1000 points per loss) + num rounds + remaining boss HP
    The fittest will be the one w/ the lowest value
    """
   
    victory_points = 0 if fight['is_victory'] else 1000
    return fight['rounds'] + fight['boss']['hp'] + victory_points

# The Individual **(~30%)**


*   Chromosone
*   Crossover
*   Mutation




## Discussion and justification on the approaches taken for the above

---

## Chromosome
The "Chromosome" of each individual is essentially the combination of its stats: hp, str, def and heal. There are some constraints to be followed for these stats:
- Each of these values has to be an integer
- the HP stats can not be lower than `Problem.min_hp` (Default to 1)
- the sum of all stats can not exceed `Problem.max_stats_points` (Default to 20)

## Crossover
The crossover to be applied is the "Arithmetic Crossover". It will be applied by combining the genetic material from the parents, calculating average values and if necessary, randomly increasing/decreasing stats until individual complies with constrainght of defined number of stats points.
The crossover can be found in classmethod `from_parents()`, which essentially generates a new individual based on the 2 individuals provided.

*Note: This final solution wasn't the one I started with. I started w/ suggestion seen in class, but the values for each individual requires integer values, it didn't fit for purpose.*


## Mutation
Mutation will be applied by randomly, and at a given chance, adding/removing from one stat and removing/adding to another stat.

---



In [ ]:
@dataclass
class Individual: 
  hp: int # health points
  strength: int
  defense: int
  healing: int
  problem: Optional[Any] = None
  is_boss: Optional[bool] = False
  cost: Optional[int] = None

  @classmethod
  def randonly_generated(cls, problem):
    new = cls(**problem.generate_random_stats(), problem=problem)
    return new

  # Initial implementation of crossover
  # which didn't work quite well for this solution
  # def from_parents(cls, prob:Problem, parent1, parent2, explore_rate_crossover):
  #   """
  #   This is the crossover function
  #   """
  #   alpha = random.uniform(-explore_rate_crossover, 1+explore_rate_crossover)
  #   attrs = ['hp', 'strength', 'defense', 'healing']
  #   child_attrs = {}
  #   for attr in attrs:
  #     child_attrs[attr] = round((alpha * getattr(parent1, attr)) + ((1-alpha) * getattr(parent2, attr)), 0)
  #   child = cls(**child_attrs, problem=problem)
  #   return child

  @classmethod
  def nivellate_stats(cls, char_stats:dict, prob:Problem):
    """
    Given a dict of char_stats and prob, 
    Make sure that provided stats are according to problem constraints.
    For that, chars can randomly be modified by added/removed stats 
    """
    attrs = ['hp', 'strength', 'defense', 'healing']
    
    # ensuring min hp constraint
    if char_stats['hp'] < prob.min_hp:
      char_stats['hp'] = prob.min_hp

    attrs_sum = sum(char_stats.values())
    while attrs_sum != prob.max_stats_points:

      # remove == -1 and add == +1
      add_or_remove = -1 if attrs_sum > prob.max_stats_points else 1

      # picking a random attr to be updated
      random_attr = random.choice(attrs)

      # if subtracting, we need to make sure no stat go below 0
      if add_or_remove == -1 and char_stats[random_attr] == 0:
          continue

      # We also need to check if HP is always above problem.min_hp
      if add_or_remove == -1 and random_attr == 'hp' and char_stats[random_attr] == prob.min_hp:
          continue

      # add or remove from random attr
      char_stats[random_attr] += add_or_remove

      # calculate sum of attrs again to break loop
      attrs_sum = sum(char_stats.values())  
      
    return char_stats

  @abstractmethod
  def mutate_stats(stats:dict, prob:Problem, params):
    """
    This abstractmethod receives a dict of stats, the problem definition 
    and the genetics params.
    It loops through all stats, and by the chance of the probabily of params.mutation_rate,
    it also by chance increase or decrease it's value by params.range_of_mutation (default to 1).
    
    At the end, to make sure the char stats are still compliant w/ the constraints,
    we call `nivellate_stats()`.

    """
    for attr in stats.keys():
      luck = random.uniform(0, 1)
      # Checking if individual was lucky to be mutated

      if luck < params.mutation_rate:
        # eg: 1 or -1
        add_or_remove = random.choice([params.range_of_mutation, -1 *params.range_of_mutation])
        stats[attr] += add_or_remove
        # Making sure mutation doesn't make negative stats
        if add_or_remove < 0 and (stats[attr] + add_or_remove) < 0:
          continue  
        # Making sure mutation does not produce chars that break problem constraint of min_hp 
        elif attr == 'hp' and add_or_remove < 0 and (stats[attr] + add_or_remove) < prob.min_hp:
          continue

      return Individual.nivellate_stats(stats, prob)


  @classmethod
  def from_parents(cls, prob:Problem, parent1, parent2, params):
    """
    This is the crossover function
    The child is a combination of values from parent1 and parent2.
    If combination of values exceeds problem.max_stats_points, stats are randomly 
    added or subtracted until constraint is satisfied.

    After that, child's attributes are mutated 
    """
    attrs = ['hp', 'strength', 'defense', 'healing']
    child_attrs = {}

    # Creating average value of parents for child attributes
    for attr in attrs:
      child_attrs[attr] = math.ceil((getattr(parent1, attr) + getattr(parent2, attr)) /2)

    child_attrs = Individual.nivellate_stats(child_attrs, prob)
      
    # run mutation
    child_attrs = Individual.mutate_stats(child_attrs, prob, params)
  
    child = cls(**child_attrs, problem=prob)
    return child
  
  def __post_init__(self):
      """
      The __post_init__ method is called after objects of type dataclasses are instantiated.
      We'll use it to make sure:
      - we don't create individuals that break problem constraint of min HP
      - we don't create individuals that break problem constraint of max stat points (unless it is a boss)
      We'll also use it to calculate the cost/fitness of every individual, as soon as it is instantiated.
      """
      
      # define max HP to avoid healing more than health
      self.max_hp = self.hp 
      self.sum_attrs = self.hp + self.strength + self.defense + self.healing

      if not self.is_boss:
        if self.hp < self.problem.min_hp:
          print(self)
          raise Exception(f"This character must have a min of {self.problem.min_hp} HP")
        
        if self.sum_attrs > self.problem.max_stats_points:
          raise Exception(f"This character exceeds the max number {self.sum_attrs}/({self.problem.max_stats_points}) of stats: {self.get_status()}")

        self.update_cost()

        
  def get_status(self): 
      return {
          'sum_stats': self.sum_attrs,
          'hp': self.hp,
          'max_hp': self.max_hp,
          'strength': self.strength,
          'defense': self.defense,
          'healing': self.healing,
          'cost': self.cost
      }

  def get_game_stats(self):
    """
    This method returns Individual stats in a more friendly way,
    to make the visualization of the example Individual more appealing 
    by using emojis to represent stats in a formated string.
    Eg: [🩸38/50|⚔️6|🛡️5|🩹4]

    """
    stats = [
        f'🩸{self.hp}/{self.max_hp}',
        f'⚔️{self.strength}',
        f'🛡️{self.defense}',
        f'🩹{self.healing}',
    ]
    return f"[{'|'.join(stats)}]"


  def self_heal(self):
    """
    Applies healing to an individual, making sure it never exceeds the max_hp
    """
    self.hp = min(self.hp + self.healing, self.max_hp)

  def reset_hp(self):
    """
    Resets chars HP. Useful for the boss, who fights many battles!
    """
    self.hp = self.max_hp

  def attack(self, target):
    """
    Used to perform an attack on an given target.
    The damage is the positive difference between atacker strength - target defense
    """
    damage = self.strength - target.defense
    if damage > 0:
      target.hp -= damage

  def battle_log(self, message:str, output=False):
    """
    This method is only a helper to print a message in relation to the battle.
    Make sure that output is False to avoid unecessary messages when running the 
    genetics algorithm.
    """

    if output:
      print(f"Battle Log: {message}")
  
  def fight(self, boss, output=False):
    """
    This method represents the actual battle between an individual and a given boss.
    The battle system is based on turns, following the order:

    - The character atacks the boss
      - The boss will have its HP deducted
      - If the boss HP is gets equal or below 0, the character wins the game
    - The boss atacks the character
      - The character will have its HP deducted
      - If the character HP is gets equal or below 0, the character loses the game
    - The healing effect is triggered for the character and for the boss
    - repeat the process until character wins or loses game (or after reaching problem.max_fight_rounds) 
    also considered a technical loss for the character.
    """

    round_number = 1
    victory = False
    reason = None
    self.battle_log(f"The Boss {boss.get_game_stats()} has a new challanger!", output)
    self.battle_log(f"The Challanger is {self.get_game_stats()}", output)

    while True:
      self.battle_log(f"Round {round_number}... Fight!", output)
      # char atacks boss
      self.battle_log(f"Char attacks boss!", output)
      self.attack(boss)
      self.battle_log(f"Boss: {boss.get_game_stats()}", output)
      if boss.hp <= 0: # boss lost
        reason = 'boss died'
        victory = True
        self.battle_log(f"Boss has died! Char wins!", output)
        break

      # boss atacks char
      self.battle_log(f"Boss attacks char!", output)
      boss.attack(self)
      self.battle_log(f"Char: {self.get_game_stats()}", output)
      if self.hp <= 0: # char lost
        reason = 'char died'
        self.battle_log(f"Char has died! Boss wins!", output)
        break

      self.battle_log(f"Both of them heal themselves!", output)
      self.self_heal()
      boss.self_heal()
      self.battle_log(f"Char {self.get_game_stats()} VS Boss {boss.get_game_stats()}", output)

      if round_number == self.problem.max_fight_rounds:
        reason = 'technical loss'
        self.battle_log(f"Char took to long to win... Technical loss, boss wins!", output)
        break
      else:
        round_number += 1

    result = {
        'is_victory': victory,
        'rounds': round_number,
        'reason': reason, 
        'char': self.get_status(),
        'boss': boss.get_status()
    }
    self.reset_hp()
    boss.reset_hp()

    return result
    
  def update_cost(self):
    """
    it simulates a fight agains the boss, and the result is used as an input for
    the problem.cost_function().
    """
    fight_result = self.fight(self.problem.boss)
    self.cost = self.problem.cost_function(fight_result)

  


### Defining the Boss & Problem
In the next cell we're going to define the Boss, which will be used also to define the problem.

In [ ]:
# Defining the boss and problem
boss = Individual(hp=50, strength=6, healing=4, defense=5, is_boss=True)
problem = Problem(boss)

## Running the algorithm  **(~10%)**





---

### Parameter choices
After many runs, it has been observed that usually the best individual has been found before 300 generations using the following parameters:
```
self.population_size = 1000
self.number_of_generations = 500
self.child_rate = 0.5
self.mutation_rate = 0.2
self.range_of_mutation = 1
```
However, only a few variations of bosses were experimented. So, despite the fact that, for most runs, the algorithm could be running unnecessarily, it was decided to leave it with 500 generations should a more harder to solve problem be inputted as input.

### Modifications (if any) to run_genetic

- As it was opted to do crossover by calculating the average of the stats of the parents, then nivellating the values, the run_genetic was modified to generate only 1 child instead of 2 children, as otherwise they could potentially be the very same.

- The `from_parents()` method from the individual already makes sure that mutation is applied during the individual "conception" as well as making sure that its cost is updated when the object is initiated.
---



In [ ]:
class Parameters:
  def __init__(self) -> None:
    self.population_size = 1000
    self.number_of_generations = 500
    self.child_rate = 0.5
    self.mutation_rate = 0.2
    self.range_of_mutation = 1

params = Parameters()

In [ ]:
def run_genetic(problem:Problem, params:Parameters):
  # read the problem
  cost_function = problem.cost_function
  # read parameters
  current_population_size = params.population_size
  num_children = params.child_rate * current_population_size
  mutation_rate = params.mutation_rate

  # setting best solution
  best_solutions = []
  best_solution = None

  # initialise population
  # population = [Individual(problem) for _ in range(params.population_size)]
  population = []
  for _ in range(current_population_size):
    new = Individual.randonly_generated(problem)
    population.append(new)
    if best_solution is None or new.cost < best_solution.cost:
      best_solution = deepcopy(new)
      best_solutions.append(best_solution)

  print(f"Created population of {len(population)} elements")
  print(f"Best solution so far has cost of {best_solution.cost}")
 
  # Generation Loop:
  for generation in range(params.number_of_generations):
    if (generation+1) % 100 == 0:
      print(f"Generation {generation+1}")
    children = []
    while len(children) < num_children:
      # randomly choosing parents
      p1, p2 = problem.gen_2_random_unique(0, len(population)-1)
      parent1, parent2 = population[p1], population[p2]
      
      # let's make some children!
      # Note: The from_parents() method automatically does the mutation 
      # and also makes sure that the cost of the Invidividual is updated.
      child1 = Individual.from_parents(problem, parent1, parent2, params)
        
      # add children to population
      children.extend([child1])
      # children.extend([child1, child2])

      # get cost of children (updated on mutation)

    # sort and cull population
    population.extend(children)
    population = sorted(population, key = lambda x: x.cost)
    population = population[:current_population_size]
     
    if population[0].cost < best_solution.cost:
      best_solution = deepcopy(population[0])
      best_solutions.append(best_solution)
      print(f"Found best solution in generation {generation+1}! {best_solution.cost}")

  print(f"In the end, best solution has cost of  {best_solution.cost}")
  
  return population, best_solution, best_solutions

In [ ]:
population, best_solution, best_solutions_over_time = run_genetic(problem, params)

Created population of 1000 elements
Best solution so far has cost of 1044
Found best solution in generation 32! 46
Found best solution in generation 44! 23
Generation 100
Generation 200
Found best solution in generation 210! 14
Found best solution in generation 243! 10
Generation 300
Generation 400
Generation 500
In the end, best solution has cost of  10


## Results and conclusions    **(~10%)**

---

This implementation of genetic algorithm has the objective of finding an individual with the best cost (fitness), which in this case was a character capable of defeating or at least causing more damage to a pre-defined boss. It uses a simplified, but similar principle of natural selection and genetics. 

The algorithm was capable of randomly generating a popupulation of individuals, generating new children, and only keeping the most fitest ones, which leads to the potentially most effective individual over the generations.

It is impressive to vizualise that from a problem "How can I create a individual that can defeat a boss?", a solution was found programmatically without any direct "clues", except by keeping the most fittest ones and evolving from them.

By working with characteristics that were not as straight forward as a list of genes, this exercise was efficient on teach me how genetic algorithms are powerful and can be used for a variety of problems.

---

In the next cells you can see more details about:
- the best solution
- how "best" individuals changed over the generations
- and a replay of a fight between the best solution and the boss



In [ ]:
# Let's see the best invidivual
print(best_solution)

Individual(hp=1, strength=13, defense=6, healing=0, problem=<__main__.Problem object at 0x7fee1058c820>, is_boss=False, cost=10)


In [ ]:
# Let's see the evolution of the best individuals
for i in best_solutions_over_time:
  print(i)

Individual(hp=8, strength=4, defense=2, healing=6, problem=<__main__.Problem object at 0x7fee105a0f10>, is_boss=False, cost=1100)
Individual(hp=7, strength=8, defense=2, healing=3, problem=<__main__.Problem object at 0x7fee105a08e0>, is_boss=False, cost=1051)
Individual(hp=3, strength=7, defense=2, healing=8, problem=<__main__.Problem object at 0x7fee10578eb0>, is_boss=False, cost=1049)
Individual(hp=4, strength=8, defense=2, healing=6, problem=<__main__.Problem object at 0x7fee1058fdf0>, is_boss=False, cost=1048)
Individual(hp=4, strength=9, defense=2, healing=5, problem=<__main__.Problem object at 0x7fee10591c10>, is_boss=False, cost=1047)
Individual(hp=5, strength=11, defense=2, healing=2, problem=<__main__.Problem object at 0x7fee10543340>, is_boss=False, cost=1044)
Individual(hp=4, strength=10, defense=3, healing=3, problem=<__main__.Problem object at 0x7fee105662b0>, is_boss=False, cost=46)
Individual(hp=3, strength=11, defense=4, healing=2, problem=<__main__.Problem object at 0x

In [ ]:
# Let's replay the battle between the best char vs the boss
battle_summary = best_solution.fight(problem.boss, output=True)

Battle Log: The Boss [🩸50/50|⚔️6|🛡️5|🩹4] has a new challanger!
Battle Log: The Challanger is [🩸1/1|⚔️13|🛡️6|🩹0]
Battle Log: Round 1... Fight!
Battle Log: Char attacks boss!
Battle Log: Boss: [🩸42/50|⚔️6|🛡️5|🩹4]
Battle Log: Boss attacks char!
Battle Log: Char: [🩸1/1|⚔️13|🛡️6|🩹0]
Battle Log: Both of them heal themselves!
Battle Log: Char [🩸1/1|⚔️13|🛡️6|🩹0] VS Boss [🩸46/50|⚔️6|🛡️5|🩹4]
Battle Log: Round 2... Fight!
Battle Log: Char attacks boss!
Battle Log: Boss: [🩸38/50|⚔️6|🛡️5|🩹4]
Battle Log: Boss attacks char!
Battle Log: Char: [🩸1/1|⚔️13|🛡️6|🩹0]
Battle Log: Both of them heal themselves!
Battle Log: Char [🩸1/1|⚔️13|🛡️6|🩹0] VS Boss [🩸42/50|⚔️6|🛡️5|🩹4]
Battle Log: Round 3... Fight!
Battle Log: Char attacks boss!
Battle Log: Boss: [🩸34/50|⚔️6|🛡️5|🩹4]
Battle Log: Boss attacks char!
Battle Log: Char: [🩸1/1|⚔️13|🛡️6|🩹0]
Battle Log: Both of them heal themselves!
Battle Log: Char [🩸1/1|⚔️13|🛡️6|🩹0] VS Boss [🩸38/50|⚔️6|🛡️5|🩹4]
Battle Log: Round 4... Fight!
Battle Log: Char attacks boss!
Battle L